In [1]:
import cv2
import os

# Video file path
video_path = '/home/kyoungsu/Desktop/20241220_135359.mp4'
output_folder = '/home/kyoungsu/Download/output/'
os.makedirs(output_folder, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the step to extract approximately 40 frames
num_frames_to_extract = 40
step = max(total_frames // num_frames_to_extract, 1)

frame_count = 0
saved_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save every 'step' frame
    if frame_count % step == 0:
        output_path = os.path.join(output_folder, f'frame_{saved_count:04d}.jpg')
        cv2.imwrite(output_path, frame)
        saved_count += 1

    frame_count += 1

    # Stop if we have saved enough frames
    if saved_count >= num_frames_to_extract:
        break

cap.release()
print(f"Saved {saved_count} frames to {output_folder}")


Saved 40 frames to /home/kyoungsu/Download/output/


In [13]:
import os
from moviepy.video.io.VideoFileClip import VideoFileClip

# Video file path
video_path = '/home/kyoungsu/Desktop/ch.mp4'
output_folder = '/home/kyoungsu/Downloads/output/'
os.makedirs(output_folder, exist_ok=True)

# Load the video file using MoviePy
clip = VideoFileClip(video_path)

# Get total number of frames based on video duration and fps
total_frames = int(clip.fps * clip.duration)

# Define the step to extract approximately 40 frames
num_frames_to_extract = 140
step = max(total_frames // num_frames_to_extract, 1)

frame_count = 0
saved_count = 0

for frame in clip.iter_frames(fps=clip.fps, dtype="uint8"):
    if frame_count % step == 0:
        output_path = os.path.join(output_folder, f'frame_{saved_count:04d}.jpg')
        # Save the frame using MoviePy's PIL integration
        from PIL import Image
        img = Image.fromarray(frame)
        img.save(output_path)
        saved_count += 1

    frame_count += 1

    # Stop if we have saved enough frames
    if saved_count >= num_frames_to_extract:
        break

clip.close()
print(f"Saved {saved_count} frames to {output_folder}")


Saved 140 frames to /home/kyoungsu/Downloads/output/


In [15]:
import os
import cv2
import numpy as np

# Helper function for camera calibration
def calibrate_camera(chessboard_images, pattern_size):
    obj_points = []
    img_points = []
    objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

    for img_path in chessboard_images:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        ret, corners = cv2.findChessboardCorners(img, pattern_size, None)
        if ret:
            img_points.append(corners)
            obj_points.append(objp)

    ret, K, D, _, _ = cv2.calibrateCamera(obj_points, img_points, img.shape[::-1], None, None)
    return K, D, obj_points, img_points

# Function to draw a cube on chessboard using explicit extrinsic parameters
def draw_cube_on_chessboard(image, corners, obj_points, K, D):
    # Compute extrinsic parameters explicitly
    objp = obj_points
    corners_reshaped = corners.reshape(-1, 2)

    # Use OpenCV's least-squares method to estimate rotation and translation
    A = []
    B = []
    for i, corner in enumerate(corners_reshaped):
        x, y = corner
        X, Y, Z = objp[i]

        # Populate the A matrix and B vector
        A.append([X, Y, Z, 1, 0, 0, 0, 0, -x*X, -x*Y, -x*Z, -x])
        A.append([0, 0, 0, 0, X, Y, Z, 1, -y*X, -y*Y, -y*Z, -y])
        B.extend([x, y])

    A = np.array(A)
    B = np.array(B)

    # Solve for extrinsics using SVD
    _, _, Vt = np.linalg.svd(A)
    extrinsics = Vt[-1].reshape(3, 4)
    R = extrinsics[:, :3]
    t = extrinsics[:, 3]

    # Ensure R is a valid rotation matrix
    U, _, Vt = np.linalg.svd(R)
    R = np.dot(U, Vt)

    # Define cube 3D points
    square_size = 0.025
    cube_points = np.float32([
        [0, 0, 0], [0, square_size, 0], [square_size, square_size, 0], [square_size, 0, 0],
        [0, 0, -square_size], [0, square_size, -square_size], [square_size, square_size, -square_size], [square_size, 0, -square_size]
    ])

    # Project 3D points onto 2D image plane
    imgpts = []
    for point in cube_points:
        p_camera = np.dot(K, np.dot(R, point) + t)
        p_image = p_camera[:2] / p_camera[2]
        imgpts.append(p_image)

    imgpts = np.int32(imgpts).reshape(-1, 2)

    # Draw the cube
    image = cv2.drawContours(image, [imgpts[:4]], -1, (0, 255, 0), 3)  # Base square
    for i, j in zip(range(4), range(4, 8)):
        image = cv2.line(image, tuple(imgpts[i]), tuple(imgpts[j]), (255, 0, 0), 3)  # Vertical lines
    image = cv2.drawContours(image, [imgpts[4:]], -1, (0, 0, 255), 3)  # Top square

    return image

# Define pattern size for calibration
pattern_size = (9, 6)  # Chessboard pattern size

# Directory containing chessboard images
image_folder = '/home/kyoungsu/Downloads/output'
chessboard_images = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.jpg')]

# Calibrate the camera
K, D, obj_points_list, img_points_list = calibrate_camera(chessboard_images, pattern_size)

# Process each image to draw the cube
output_folder = 'output_with_cubes'
os.makedirs(output_folder, exist_ok=True)

for img_path, obj_points, img_points in zip(chessboard_images, obj_points_list, img_points_list):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

    if ret:
        img = draw_cube_on_chessboard(img, corners, obj_points, K, D)
        output_path = os.path.join(output_folder, os.path.basename(img_path))
        cv2.imwrite(output_path, img)
        print(f"Cube drawn on {img_path} and saved to {output_path}")
    else:
        print(f"Chessboard not found in {img_path}")

print("Processing complete.")


Chessboard not found in /home/kyoungsu/Downloads/output/frame_0100.jpg
Chessboard not found in /home/kyoungsu/Downloads/output/frame_0132.jpg
Chessboard not found in /home/kyoungsu/Downloads/output/frame_0084.jpg
Chessboard not found in /home/kyoungsu/Downloads/output/frame_0062.jpg
Processing complete.
